In [1]:
# If needed (uncomment the next 2 lines in a clean env):
# %pip install pandas numpy python-dateutil holidays

import pandas as pd
import numpy as np
from datetime import datetime
import holidays

# ---- Config (tweak as needed) ----
# COUNTRY = "US"           # "SG" for Singapore, e.g. "US" for United States
CSV_PATH = "Data-GP1.csv"
DATE_FMT = "%m/%d/%y"    # your sample looks like mm/dd/yy


In [2]:
# Read, parse dates, and do light cleanup
df = pd.read_csv(
    CSV_PATH,
    parse_dates=["Dept_Date", "Purchase_Date"],
    date_parser=lambda s: pd.to_datetime(s, format=DATE_FMT, errors="coerce")
)

# Standardize column names (optional, but makes life easier)
df.columns = [c.strip() for c in df.columns]

# Coerce “flag” columns to int/bool where appropriate
flag_cols = ["isNormCabin", "isReturn", "isOneway"]
for c in flag_cols:
    if c in df.columns:
        df[c] = df[c].astype(int)

df.head()


/tmp/ipykernel_1095423/414417218.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(


,num_seats_total,mean_net_ticket_price,Dept_Date,Purchase_Date,Train_Number_All,Culmulative_sales,isNormCabin,isReturn,isOneway,Customer_Cat
0,1,188.663422,2019-10-09,2018-10-10,A,1,1,0,0,A
1,1,358.428869,2019-05-27,2018-06-06,B,1,0,1,0,A
2,1,356.372684,2020-05-31,2019-06-17,B,1,0,1,0,A
3,1,396.370505,2019-12-10,2018-12-27,C,1,0,1,0,A
4,1,364.264552,2020-05-31,2019-06-19,B,2,0,1,0,A


In [4]:
# # Build holiday calendar for the chosen country
# try:
#     holiday_cal = holidays.country_holidays(COUNTRY)
# except Exception as e:
#     print(f"Falling back to no-holiday mode due to: {e}")
#     holiday_cal = set()

# # Helper to test holiday
# def is_holiday(d):
#     if pd.isna(d):
#         return False
#     return d in holiday_cal

# Core features:
df["dept_weekday"]     = df["Dept_Date"].dt.day_name()
df["purchase_weekday"] = df["Purchase_Date"].dt.day_name()
# df["is_holiday_dept"]  = df["Dept_Date"].apply(is_holiday).astype(int)
# df["is_holiday_purchase"] = df["Purchase_Date"].apply(is_holiday).astype(int)
df["lead_time_days"]   = (df["Dept_Date"] - df["Purchase_Date"]).dt.days

# Useful calendar features (often boost tree models)
df["dept_year"]   = df["Dept_Date"].dt.year
df["dept_month"]  = df["Dept_Date"].dt.month
df["dept_day"]    = df["Dept_Date"].dt.day
df["dept_week"]   = df["Dept_Date"].dt.isocalendar().week.astype(int)
df["dept_quarter"]= df["Dept_Date"].dt.quarter

df["purchase_year"]  = df["Purchase_Date"].dt.year
df["purchase_month"] = df["Purchase_Date"].dt.month
df["purchase_week"]  = df["Purchase_Date"].dt.isocalendar().week.astype(int)
df["purchase_quarter"]= df["Purchase_Date"].dt.quarter

# Weekend flags
df["is_weekend_dept"]     = df["Dept_Date"].dt.weekday.isin([5,6]).astype(int)
df["is_weekend_purchase"] = df["Purchase_Date"].dt.weekday.isin([5,6]).astype(int)

df[[
    "Dept_Date","Purchase_Date","dept_weekday","purchase_weekday",
    "lead_time_days",
    "is_weekend_dept","is_weekend_purchase"
]].head()


,Dept_Date,Purchase_Date,dept_weekday,purchase_weekday,lead_time_days,is_weekend_dept,is_weekend_purchase
0,2019-10-09,2018-10-10,Wednesday,Wednesday,364,0,0
1,2019-05-27,2018-06-06,Monday,Wednesday,355,0,0
2,2020-05-31,2019-06-17,Sunday,Monday,349,1,0
3,2019-12-10,2018-12-27,Tuesday,Thursday,348,0,0
4,2020-05-31,2019-06-19,Sunday,Wednesday,347,1,0


In [5]:
import holidays

# Build holiday calendars
sg_holidays = holidays.country_holidays("SG")
us_holidays = holidays.country_holidays("US")
cn_holidays = holidays.country_holidays("CN")
fr_holidays = holidays.country_holidays("FR")
in_holidays = holidays.country_holidays("IN")
ru_holidays = holidays.country_holidays("RU")

# Helper
def is_holiday_in(calendar, d):
    if pd.isna(d):
        return 0
    return int(d in calendar)

# Add holiday flags for Dept_Date
df["is_sg_holiday_dept"] = df["Dept_Date"].apply(lambda d: is_holiday_in(sg_holidays, d))
df["is_us_holiday_dept"] = df["Dept_Date"].apply(lambda d: is_holiday_in(us_holidays, d))
df["is_cn_holiday_dept"] = df["Dept_Date"].apply(lambda d: is_holiday_in(cn_holidays, d))
df["is_fr_holiday_dept"] = df["Dept_Date"].apply(lambda d: is_holiday_in(fr_holidays, d))
df["is_in_holiday_dept"] = df["Dept_Date"].apply(lambda d: is_holiday_in(in_holidays, d))
df["is_ru_holiday_dept"] = df["Dept_Date"].apply(lambda d: is_holiday_in(ru_holidays, d))

# Add holiday flags for Purchase_Date
df["is_sg_holiday_purchase"] = df["Purchase_Date"].apply(lambda d: is_holiday_in(sg_holidays, d))
df["is_us_holiday_purchase"] = df["Purchase_Date"].apply(lambda d: is_holiday_in(us_holidays, d))
df["is_cn_holiday_purchase"] = df["Purchase_Date"].apply(lambda d: is_holiday_in(cn_holidays, d))
df["is_fr_holiday_purchase"] = df["Purchase_Date"].apply(lambda d: is_holiday_in(fr_holidays, d))
df["is_in_holiday_purchase"] = df["Purchase_Date"].apply(lambda d: is_holiday_in(in_holidays, d))
df["is_ru_holiday_purchase"] = df["Purchase_Date"].apply(lambda d: is_holiday_in(ru_holidays, d))

df[[
    "Dept_Date","Purchase_Date",
    "is_sg_holiday_dept","is_us_holiday_dept","is_cn_holiday_dept","is_fr_holiday_dept","is_in_holiday_dept","is_ru_holiday_dept",
    "is_sg_holiday_purchase","is_us_holiday_purchase","is_cn_holiday_purchase","is_fr_holiday_purchase","is_in_holiday_purchase","is_ru_holiday_purchase"
]].head()


,Dept_Date,Purchase_Date,is_sg_holiday_dept,is_us_holiday_dept,is_cn_holiday_dept,is_fr_holiday_dept,is_in_holiday_dept,is_ru_holiday_dept,is_sg_holiday_purchase,is_us_holiday_purchase,is_cn_holiday_purchase,is_fr_holiday_purchase,is_in_holiday_purchase,is_ru_holiday_purchase
0,2019-10-09,2018-10-10,0,0,0,0,0,0,0,0,0,0,0,0
1,2019-05-27,2018-06-06,0,1,0,0,0,0,0,0,0,0,0,0
2,2020-05-31,2019-06-17,0,0,0,0,0,0,0,0,0,0,0,0
3,2019-12-10,2018-12-27,0,0,0,0,0,0,0,0,0,0,0,0
4,2020-05-31,2019-06-19,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
# Price-related transforms
df["log_price"] = np.log1p(df["mean_net_ticket_price"])

# Interaction features
df["return_x_norm"]   = df["isReturn"] * df["isNormCabin"]
df["oneway_x_norm"]   = df["isOneway"] * df["isNormCabin"]
df["return_x_weekend"]= df["isReturn"] * df["is_weekend_dept"]
df["oneway_x_weekend"]= df["isOneway"] * df["is_weekend_dept"]

# Demand proxy features
# (Culmulative_sales appears to be a running count per Train_Number_All; we can create rank/order and recent growth signals)
if {"Train_Number_All","Culmulative_sales"}.issubset(df.columns):
    df = df.sort_values(["Train_Number_All", "Dept_Date", "Purchase_Date"])
    df["route_sales_rank"] = df.groupby("Train_Number_All")["Culmulative_sales"].rank(method="first").astype(int)
    # A naive short-window diff (by train) as a "recent growth" proxy
    df["sales_diff_1"] = df.groupby("Train_Number_All")["Culmulative_sales"].diff(1).fillna(0)

# Seasonality buckets
def season_from_month(m):
    # Simple Northern-hemisphere style seasons; tweak if you want SG wet/dry seasons instead
    if m in (12,1,2): return "Winter"
    if m in (3,4,5):  return "Spring"
    if m in (6,7,8):  return "Summer"
    return "Autumn"

df["dept_season"] = df["dept_month"].apply(season_from_month)

# Relative price within (route, month) — helps normalize price levels across routes
df["route_month_mean_price"] = df.groupby(["Train_Number_All","dept_year","dept_month"])["mean_net_ticket_price"].transform("mean")
df["price_to_route_month_mean"] = df["mean_net_ticket_price"] / df["route_month_mean_price"]

df.head()


,num_seats_total,mean_net_ticket_price,Dept_Date,Purchase_Date,Train_Number_All,Culmulative_sales,isNormCabin,isReturn,isOneway,Customer_Cat,...,log_price,return_x_norm,oneway_x_norm,return_x_weekend,oneway_x_weekend,route_sales_rank,sales_diff_1,dept_season,route_month_mean_price,price_to_route_month_mean
47196,1,418.084961,2018-06-01,2018-06-01,A,1,0,0,0,A,...,6.038074,0,0,0,0,1,0.0,Summer,399.808336,1.045713
207766,1,464.041601,2018-06-02,2018-06-01,A,1,0,0,0,B,...,6.142127,0,0,0,0,2,0.0,Summer,399.808336,1.160660
44629,4,561.181241,2018-06-03,2018-06-01,A,4,0,0,0,A,...,6.331824,0,0,0,0,4632,3.0,Summer,399.808336,1.403626
46277,1,633.715657,2018-06-03,2018-06-02,A,5,0,0,0,A,...,6.453177,0,0,0,0,5509,1.0,Summer,399.808336,1.585049
208931,2,763.550821,2018-06-03,2018-06-03,A,2,0,0,1,B,...,6.639288,0,0,0,1,2134,-3.0,Summer,399.808336,1.909792


In [7]:
OUT_PATH = "Data-GP1_with_features.csv"
df.to_csv(OUT_PATH, index=False)
print(f"Saved: {OUT_PATH}\nRows: {len(df)} | Columns: {df.shape[1]}")



Saved: Data-GP1_with_features.csv
Rows: 209697 | Columns: 46


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 209697 entries, 47196 to 192898
Data columns (total 46 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   num_seats_total            209697 non-null  int64         
 1   mean_net_ticket_price      209697 non-null  float64       
 2   Dept_Date                  209697 non-null  datetime64[ns]
 3   Purchase_Date              209697 non-null  datetime64[ns]
 4   Train_Number_All           209697 non-null  object        
 5   Culmulative_sales          209697 non-null  int64         
 6   isNormCabin                209697 non-null  int64         
 7   isReturn                   209697 non-null  int64         
 8   isOneway                   209697 non-null  int64         
 9   Customer_Cat               209697 non-null  object        
 10  dept_weekday               209697 non-null  object        
 11  purchase_weekday           209697 non-null  object   